In [2]:
import pandas as pd

In [215]:
#input cap table
investors = ['Founders', 'Omnes Capital', 'CM-CIC', 'Cap Decisif', 'Sofinnova', 'Valquest', 'EIF']
series = ['Common shares', 'A shares', 'A2 shares', 'B shares', 'C shares']
shares = [
    [100,0,0,0,30],
    [0,40,30,20,10],
    [0,20,10,5,4],
    [0,0,10,5,2],
    [0,0,0,50,30],
    [0,0,0,30,0],
    [0,0,0,10,0],
    ]
prices = [1,20,30,40,50]

#cap table init
cap_table = pd.DataFrame(shares, index = investors, columns = series)

#Columns total
for i in range (len(series)):
    cap_table.loc["Total", series[i]] = cap_table[series[i]].sum()

#Lines total
for i in range (len(investors)):
    cap_table.loc[investors[i], "Total shares"] = cap_table.loc[investors[i],].sum()
    
#Grand total
cap_table.loc["Total","Total shares"] = cap_table.loc["Total",].sum()

#% column
cap_table["Total shares (%)"] = cap_table["Total shares"] / cap_table.at["Total", "Total shares"]

#format
format_dict = { i : '{:,.0f}'.format for i in list(cap_table.columns)[:-1] }
format_dict["Total shares (%)"] = '{:,.2%}'.format

cap_table.style.format(format_dict)

,Common shares,A shares,A2 shares,B shares,C shares,Total shares,Total shares (%)
Founders,100,0,0,0,30,130,32.02%
Omnes Capital,0,40,30,20,10,100,24.63%
CM-CIC,0,20,10,5,4,39,9.61%
Cap Decisif,0,0,10,5,2,17,4.19%
Sofinnova,0,0,0,50,30,80,19.70%
Valquest,0,0,0,30,0,30,7.39%
EIF,0,0,0,10,0,10,2.46%
Total,100,60,50,120,76,406,100.00%


In [217]:
#input liquid pref
first_step = "Nominal"
Galion_rate = 20
first_step_item = first_step + (first_step == "Galion")*(": " + str(Galion_rate) + "%")
liquid_pref_steps = [first_step_item,'Pref C', 'Pref B', 'Pref A2', 'Pref A', 'Prorata']
multiples_pref = [1,1,1,1]
participating = True
sale_price = 30000

#liquid pref init
liquid_pref = pd.DataFrame(index = investors+["Total"], columns = liquid_pref_steps)
solde = []
price_step_list = []


#First step
if (first_step == "Nominal"):
    price_step = prices[0]
    price_step_list.append(price_step)
    liquid_pref[liquid_pref_steps[0]] = cap_table["Total shares"] * price_step
elif (first_step == "Galion"):
    price_step = Galion_rate/100 * sale_price / cap_table.at["Total", "Total shares"]
    price_step_list.append(price_step)
    liquid_pref[liquid_pref_steps[0]] = cap_table["Total shares"] * price_step
else:
    print("ERROR FIRST STEP")
solde.append(sale_price - liquid_pref.at["Total", liquid_pref_steps[0]])


#Pref
for i in range (len(liquid_pref_steps)-2):
    price_step = max(0, multiples_pref[i]*prices[-1-i]-price_step_list[0])
    price_step_list.append(price_step)
    if (solde[-1] >= (cap_table.at["Total", series[-1-i]] * price_step)):
        liquid_pref[liquid_pref_steps[i+1]] = cap_table[series[-1-i]] * price_step
    else:
        liquid_pref[liquid_pref_steps[i+1]] = cap_table[series[-1-i]] * solde[-1] / cap_table.at["Total", series[-1-i]]
    solde.append(solde[-1] - liquid_pref.at["Total", liquid_pref_steps[i+1]])
    
#prorata
if (participating):
    price_step = solde[-1] / cap_table.at["Total", "Total shares"]
    price_step_list.append(price_step)
    liquid_pref[liquid_pref_steps[-1]] = cap_table["Total shares (%)"] * solde[-1]
    solde.append(solde[-1] - liquid_pref.at["Total", liquid_pref_steps[-1]])
else:
    print("ERROR")

    
#Lines total
for i in range (len(investors)):
    liquid_pref.loc[investors[i], "Total proceeds"] = liquid_pref.loc[investors[i],].sum()
    
#Grand total
liquid_pref.loc["Total","Total proceeds"] = liquid_pref.loc["Total",].sum()

#solde + price_step
for i in range (len(liquid_pref_steps)):
    liquid_pref.loc["Solde", liquid_pref_steps[i]] = solde[i]
    liquid_pref.loc["Prix", liquid_pref_steps[i]] = price_step_list[i] 

    
# vs prorata / delta
for i in range (len(investors)):
    liquid_pref.loc[investors[i], "vs prorata"] = cap_table.at[investors[i], "Total shares (%)"] * sale_price
    liquid_pref.loc[investors[i], "% delta"] = liquid_pref.at[investors[i], "Total proceeds"] / liquid_pref.at[investors[i], "vs prorata"] - 1
liquid_pref.loc["Total", "vs prorata"] = cap_table.at["Total", "Total shares (%)"] * sale_price
liquid_pref.loc["Total", "% delta"] = liquid_pref.at["Total", "Total proceeds"] / liquid_pref.at["Total", "vs prorata"] - 1


#format
format_dict = { i : '{:,.2f} €'.format for i in list(liquid_pref.columns)[:-1]}
format_dict["% delta"] = '{:,.2%}'.format

liquid_pref.style.format(format_dict)

,Nominal,Pref C,Pref B,Pref A2,Pref A,Prorata,Total proceeds,vs prorata,% delta
Founders,130.00 €,"1,470.00 €",0.00 €,0.00 €,0.00 €,"5,955.67 €","7,555.67 €","9,605.91 €",-21.34%
Omnes Capital,100.00 €,490.00 €,780.00 €,870.00 €,760.00 €,"4,581.28 €","7,581.28 €","7,389.16 €",2.60%
CM-CIC,39.00 €,196.00 €,195.00 €,290.00 €,380.00 €,"1,786.70 €","2,886.70 €","2,881.77 €",0.17%
Cap Decisif,17.00 €,98.00 €,195.00 €,290.00 €,0.00 €,778.82 €,"1,378.82 €","1,256.16 €",9.76%
Sofinnova,80.00 €,"1,470.00 €","1,950.00 €",0.00 €,0.00 €,"3,665.02 €","7,165.02 €","5,911.33 €",21.21%
Valquest,30.00 €,0.00 €,"1,170.00 €",0.00 €,0.00 €,"1,374.38 €","2,574.38 €","2,216.75 €",16.13%
EIF,10.00 €,0.00 €,390.00 €,0.00 €,0.00 €,458.13 €,858.13 €,738.92 €,16.13%
Total,406.00 €,"3,724.00 €","4,680.00 €","1,450.00 €","1,140.00 €","18,600.00 €","30,000.00 €","30,000.00 €",0.00%
Solde,"29,594.00 €","25,870.00 €","21,190.00 €","19,740.00 €","18,600.00 €",0.00 €,nan €,nan €,nan%
Prix,1.00 €,49.00 €,39.00 €,29.00 €,19.00 €,45.81 €,nan €,nan €,nan%


In [127]:
### A ajouter
    # options
    # Participating / non participating
    # Graphe
    # Interface user
